# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,name,lat,lon,temp_max,humidity,all,speed,country,dt
0,0,cabedelo,-6.9811,-34.8339,304.52,67,52,4.75,BR,1709743133
1,1,cururupu,-1.8283,-44.8683,301.30,86,76,2.43,BR,1709743134
2,2,hamilton,39.1834,-84.5333,288.84,97,100,5.14,US,1709742840
3,3,port-aux-francais,-49.3500,70.2167,280.23,85,97,10.05,TF,1709743136
4,4,enewetak,11.3474,162.3373,299.44,80,47,10.84,MH,1709743140


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:


# Configure the map plot
map_plot = city_data_df.hvplot.points ('lon', 
                                       'lat',
                                       color = 'name',
                                       size = 'humidity', 
                                       frame_width = 700, 
                                       frame_height = 500)

# Display the map
map_plot

:Points   [lon,lat]   (name,humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc [city_data_df ['temp_max'] > 294.15]
city_data_df = city_data_df.loc [city_data_df ['temp_max'] < 300.15]
city_data_df = city_data_df.loc [city_data_df ['speed'] < 4.5]
city_data_df = city_data_df.loc [city_data_df ['all'] == 0]

# Drop any rows with null values
city_data_df = city_data_df.dropna (how = 'any', axis = 0)

# Display sample data
city_data_df.head ()

,City_ID,name,lat,lon,temp_max,humidity,all,speed,country,dt
11,11,pisco,-13.7000,-76.2167,299.83,69,0,3.60,PE,1709743128
90,90,encarnacion de diaz,21.5167,-102.2333,295.58,18,0,4.19,MX,1709743225
98,98,lazaro cardenas,17.9583,-102.2000,299.78,60,0,2.23,MX,1709743232
132,132,minas,-34.3759,-55.2377,298.42,26,0,0.42,UY,1709743262
250,250,rio grande,-32.0350,-52.0986,298.21,53,0,4.03,BR,1709743198


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.drop (['temp_max',
                            'all',
                            'speed',
                            'dt'], axis = 1)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df ['Hotel Name'] = ''

# Display sample data
hotel_df.head ()

,City_ID,name,lat,lon,humidity,country,Hotel Name
11,11,pisco,-13.7000,-76.2167,69,PE,
90,90,encarnacion de diaz,21.5167,-102.2333,18,MX,
98,98,lazaro cardenas,17.9583,-102.2000,60,MX,
132,132,minas,-34.3759,-55.2377,26,UY,
250,250,rio grande,-32.0350,-52.0986,53,BR,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
params = {'radius': 10000,
        'categories': 'accommodation.hotel',
        'apikey': geoapify_key}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
#for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
lat = hotel_df ['lat']
lon = hotel_df ['lon']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
params = {'radius': 10000,
          'categories': 'hotel',
          'apikey': geoapify_key,
          'filter': [lon, lat, 'radius'],
          'bias': [lon, lat]}
    
    # Set base URL
base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
address = 'Av. Las Americas, Urb. Residencial Panacas B-2, Pisco'

name_address = requests.get (f'https://api.geoapify.com/v1/geocode/search?text={address}&limit=1&apiKey={geoapify_key}')
    
    # Convert the API response to JSON format
name_address = name_address.json ()

print (name_address)

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
index = 0

try:
    hotel_df.loc[index, "Hotel Name"] = name_address['features'][0]['properties']['name']
except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
    hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
print(f"{hotel_df.loc[index, 'name']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-76.219024, -13.715538]}, 'properties': {'country_code': 'pe', 'country': 'Peru', 'county': 'Pisco', 'datasource': {'sourcename': 'whosonfirst', 'url': 'https://www.whosonfirst.org/docs/licenses/'}, 'state': 'Ica', 'city': 'Pisco', 'state_code': 'ICA', 'lon': -76.219024, 'lat': -13.715538, 'result_type': 'city', 'formatted': 'Pisco, ICA, Peru', 'address_line1': 'Pisco', 'address_line2': 'ICA, Peru', 'timezone': {'name': 'America/Lima', 'offset_STD': '-05:00', 'offset_STD_seconds': -18000, 'offset_DST': '-05:00', 'offset_DST_seconds': -18000}, 'plus_code': '57R57QMJ+Q9', 'plus_code_short': 'MJ+Q9 Pisco, Peru', 'rank': {'popularity': 1.484337740538332, 'confidence': 0, 'confidence_city_level': 0, 'match_type': 'full_match'}, 'place_id': '5181423d7d040e53c0592b172aff5a6e2bc0c00208e2031e77686f736f6e66697273743a6c6f63616c6974793a343231323031393039'}, 'bbox': [-7

,City_ID,name,lat,lon,humidity,country,Hotel Name
11,11.0,pisco,-13.7000,-76.2167,69.0,PE,
90,90.0,encarnacion de diaz,21.5167,-102.2333,18.0,MX,
98,98.0,lazaro cardenas,17.9583,-102.2000,60.0,MX,
132,132.0,minas,-34.3759,-55.2377,26.0,UY,
250,250.0,rio grande,-32.0350,-52.0986,53.0,BR,
267,267.0,howrah,22.5892,88.3103,64.0,IN,
300,300.0,calama,-22.4667,-68.9333,15.0,CL,
336,336.0,chui,-33.6971,-53.4616,40.0,UY,
346,346.0,kailua-kona,19.6406,-155.9956,79.0,US,
353,353.0,minab,27.1467,57.0801,67.0,IR,


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE